In [1]:
!pip install langchain

In [1]:
!pip install langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [45]:
from langchain_community.document_loaders import PyPDFLoader
filepath="https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf"
loader=PyPDFLoader(filepath)
docs=loader.load()


In [46]:
len(docs)

11

In [47]:
# print(f"{docs[0].page_content[:200]}\n")
print(docs[0].page_content)

Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring signiﬁcantly
less time to train. Our model 

In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,chunk_overlap=200,add_start_index=True
)
all_splits=splitter.split_documents(docs)
len(all_splits)

43

In [49]:
all_splits[:3]

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [50]:
!pip install langchain_google_genai

In [23]:
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"]=getpass.getpass()
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [28]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[-0.017952103167772293, 0.02014649473130703, 0.012442673556506634, -0.07666034251451492, -0.007453585974872112, -0.00031448289519175887, -0.0009973814012482762, 0.006773720495402813, 0.017003163695335388, 0.010947381146252155]


In [29]:
!pip install langchain_core

In [30]:
from langchain_core.vectorstores import InMemoryVectorStore
vector_store=InMemoryVectorStore(embeddings)

In [31]:
ids = vector_store.add_documents(documents=all_splits)

['a7bf90ea-a7a1-452d-9c5f-1275efaa802f',
 '118d0ec5-7193-42e2-8887-50ef1e186b5a',
 'c2cd2038-18c2-4fb9-9b31-819848a8cb3d']

In [44]:
results = vector_store.similarity_search(
    "what is the blue score"
)
print(results[0])

page_content='Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the
English-to-German and English-to-French newstest2014 tests at a fraction of the training cost.
Model
BLEU Training Cost (FLOPs)
EN-DE EN-FR EN-DE EN-FR
ByteNet [15] 23.75
Deep-Att + PosUnk [32] 39.2 1.0 ·1020
GNMT + RL [31] 24.6 39.92 2.3 ·1019 1.4 ·1020
ConvS2S [8] 25.16 40.46 9.6 ·1018 1.5 ·1020
MoE [26] 26.03 40.56 2.0 ·1019 1.2 ·1020
Deep-Att + PosUnk Ensemble [32] 40.4 8.0 ·1020
GNMT + RL Ensemble [31] 26.30 41.16 1.8 ·1020 1.1 ·1021
ConvS2S Ensemble [8] 26.36 41.29 7.7 ·1019 1.2 ·1021
Transformer (base model) 27.3 38.1 3.3 · 1018
Transformer (big) 28.4 41.0 2.3 ·1019
Label Smoothing During training, we employed label smoothing of value ϵls = 0.1 [30]. This
hurts perplexity, as the model learns to be more unsure, but improves accuracy and BLEU score.
6 Results
6.1 Machine Translation
On the WMT 2014 English-to-German translation task, the big transformer model (Transform